## Word2Vec 실습하기

### 1. English Word2Vec 만들기
-------------------

- gensim 라이브러리를 이용한 word2vec 만들기
- 영어로 된 말뭉치(corpus)를 다운로드 받은 후 전처리하고, 상기 전처리된 데이터를 바탕으로 word2vec 을 만든다

In [1]:
# 필요한 라이브러리 로딩
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

##### 1) 훈련데이터 이해하기

In [9]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="../../data/ted_en-20160408.xml")

('../../data/ted_en-20160408.xml', <http.client.HTTPMessage at 0x40ac9d4210>)

- \<content>\</content> 사이에 위치한 데이터만 추출
-  (Laughter), (Applause) 와 같은 배경음을 나타내는 단어 제거

##### 2) 훈련 데이터 전처리하기

In [2]:
targetXML = open('../../data/ted_en-20160408.xml', 'r', encoding='UTF8')
targetText = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parseText = '\n'.join(targetText.xpath('//content/text()'))

In [3]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
contentText = re.sub(r'\([^)]*\)', '', parseText)

In [4]:
contentText[:500]

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit. I'm actually old enough to remember them. Facit was a fantastic company. They were born deep in the Swedish forest, and they made the best mechanical calculators in the world. Everybody used them. A"

In [5]:
print(len(contentText))

24062319


In [ ]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sentenceText = sent_tokenize(contentText)

In [ ]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
from tqdm import tqdm

normalizedText = []
for string in tqdm(sentenceText):
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalizedText.append(tokens)

In [ ]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
result = [word_tokenize(sentence) for sentence in normalizedText]

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

In [ ]:
for line in result[:3]:
    print(line)

##### 3) Word2Vec 학습(훈련) 시키기

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size = 100, window=5, min_count=5, workers=4, sg=0) 

* Word2Vec의 Hyper-parameter 값은 다음과 같습니다.

    - vector_size = 워드 벡터의 특징 값. 즉, 임베딩된 벡터의 차원
    - window = 컨텍스트 윈도우 크기
    - min_count = 단어 최소 빈도수 제한(빈도가 적은 단어들은 제외)
    - workers = 학습을 위한 프로세스 수
    - sg = 0은 CBOW, 1은 Skip-gram

- model.wv.most_similar :: 단어 유사도 계산

In [ ]:
model_result = model.wv.most_similar('man')
print(model_result)

##### 4) Word2Vec 모델 저장하고 로드하기

In [ ]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장

In [ ]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)